In [12]:
n = 6
init_tellers = 10 # A0
cost_salary = 10 #CS
cost_retire = 30 #CR
cost_hire = 10 #CH
retire_max = 10 #RM
max_labor = 160 #MH

hours_created = [4800 6000 6500 5000 4500 3500] #HrsReq (work created in each month)
max_hrs_left = [800 700 600 500 400 0]; #Lt (max hours left over at the end of each month)

In [15]:
using JuMP, Cbc

m = Model()

@variable(m, x[1:n] >= 0) # of hired employees in period t
@variable(m, y[1:n] >= 0) # of retired employees in period t
@variable(m, z[0:n] >= 0) # of current employees in period t
@variable(m, s[0:n] >= 0) # of carryover hours in period t
@variable(m, a[1:n] >= 0) # of individual's working hours in period t

@objective(m, Min, sum(cost_salary * z) + sum(cost_retire * y) + sum(cost_hire * x))

@constraint(m, curr_work[t in 1:n], z[t] == z[t-1] + x[t] - y[t])
@constraint(m, carryover[t in 1:6], s[t] == s[t-1] + (hours_created[t] - (a[t] * z[t])))
#@constraint(m, carryover[t in 1:6], s[t] == (hours_created[t] - (a[t] * z[t])))
@constraint(m, start_worker, z[0] == init_tellers)
@constraint(m, start_carryover, s[0] == 0)
@constraint(m, max_retire, sum(y) <= retire_max)
@constraint(m, max_indv_labor[t in 1:6], a[t] <= max_labor)
@constraint(m, max_carryover[t in 1:6], s[t] <= max_hrs_left[t])
@constraint(m, tot_hrs_meet, sum(a[t] * z[t] + s[t] for t in 1:6)== sum(hours_created))

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

LoadError: MathOptInterface.UnsupportedConstraint{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.EqualTo{Float64}}: `MathOptInterface.ScalarQuadraticFunction{Float64}`-in-`MathOptInterface.EqualTo{Float64}` constraint is not supported by the model.